## MLFlow Inference with Wallaroo Tutorial

Wallaroo users can upload their trained [MLFlow ML Models](https://www.mlflow.org/docs/latest/models.html) into their Wallaroo instance and perform inferences with it through a Wallaroo pipeline.

The following tutorial is a brief example of how:

* Upload a MLFlow model into your Wallaroo workspace.
* Add the MLFlow model as a step in a Wallaroo pipeline.
* Perform inferences using submitted data to a deployed pipeline.

This tutorial assumes that you have a Wallaroo instance and are running this Notebook from the Wallaroo Jupyter Hub service. This tutorial provides the following:

* `statsmodels-test`: A statsmodel ML model in MLFlow format.
* `statsmodels-test-postprocess`: A post-processing model.

### MLFlow Models and Wallaroo

MLFlow models are composed of two parts:  the model, and the flavors.  When submitting a MLFlow model to Wallaroo, both aspects must be part of the ML Model included in the container.  For full information about MLFlow model structure, see the [MLFLow Documentation](https://www.mlflow.org/docs/latest/index.html).

Wallaroo registers the models as Docker containers.  Organizations will either have to make their containers available in a public Docker or through a private container registry service.  For examples on setting up a private container registry service, see the [Docker Documentation "Deploy a registry server"](https://docs.docker.com/registry/deploying/).  For more details on setting up a container registry in a cloud environment, see the related documentation for your preferred cloud provider:
  * [Google Cloud Platform Container Registry](https://cloud.google.com/container-registry)
  * [Amazon Web Services Elastic Container Registry](https://docs.aws.amazon.com/AmazonECR/latest/userguide/what-is-ecr.html)
  *  [Microsoft Azure Container Registry](https://azure.microsoft.com/en-us/free/container-registry/)

### Prerequisites

Before uploading and running an inference with a MLFlow model in Wallaroo the following will be needed:

* **MLFlow Input Schema**:  The input schema with the fields and data types for each MLFLow model type uploaded to Wallaroo.  In the examples below, the data types are imported using the `pyarrow` library.

## MLFLow Inference Steps

To upload a MLFlow ML Model into Wallaroo, use the following general step:

* Import Libraries
* Connect to Wallaroo
* Set MLFlow Input Schemas
* Register MLFlow Model
* Create Pipeline and Add Model Steps
* Run Inference

### Import Libraries

We start by importing the libraries we will need to connect to Wallaroo and use our MLFlow models. This includes the `wallaroo` libraries, `pyarrow` for data types, and the `json` library for handling JSON data.

In [1]:
import uuid
import json

import wallaroo
from wallaroo.object import EntityNotFoundError

import pyarrow as pa

### Connect to Wallaroo

Connect to Wallaroo and store the connection in the variable `wl`.

The folowing methods are used to create the workspace and pipeline for this tutorial.  A workspace is created and set as the current workspace that will contain the registered models and pipelines.

In [2]:
wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://magical-bear-3782.keycloak.wallaroo.community/auth/realms/master/device?user_code=RCXI-FEGJ

Login successful!


In [3]:
def get_workspace(name):
    wl = wallaroo.Client()
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    wl = wallaroo.Client()
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [4]:
prefix = 'statsmodels-test'
workspace_name= f"{prefix}-workspace-{uuid.uuid4()}"
pipeline_name = f"{prefix}-{uuid.uuid4()}"

mlflowworkspace = get_workspace(workspace_name)
wl.set_current_workspace(mlflowworkspace)


pipeline = get_pipeline(pipeline_name)

### Set MLFlow Input Schemas

Set the MLFlow input schemas through the `pyarrow` library.  In the examples below, the input schemas for both the MLFlow model `statsmodels-test` and the `statsmodels-test-postprocess` model.

In [5]:
sm_input_schema = pa.schema([
  pa.field('temp', pa.float32()),
  pa.field('holiday', pa.uint8()),
  pa.field('workingday', pa.uint8()),
  pa.field('windspeed', pa.float32())
])

pp_input_schema = pa.schema([
    pa.field('predicted_mean', pa.float32())
])

### Register MLFlow Model

Use the `register_model_image` method to register the Docker container containing the MLFlow models.

In [6]:
sm_model = wl.register_model_image(
    name=f"{prefix}-statmodels",
    image=f"proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mlflow-statsmodels-example:21cfff89"
).configure("mlflow", input_schema=sm_input_schema, output_schema=pp_input_schema)
pp_model = wl.register_model_image(
    name=f"{prefix}-postprocess",
    image=f"proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mlflow-postprocess-example:ca3bcecb"
).configure("mlflow", input_schema=pp_input_schema, output_schema=pp_input_schema)

### Create Pipeline and Add Model Steps

With the models registered, we can add the MLFlow models as steps in the pipeline.  Once ready, we will deploy the pipeline so it is available for submitting data for running inferences.

In [7]:
pipeline.add_model_step(sm_model)
pipeline.add_model_step(pp_model)

name,statsmodels-test-364c7a5c-8754-48c8-aaf0-2cc055a8d869
created,2022-09-22 14:33:00.562180+00:00
last_updated,2022-09-22 14:33:00.562180+00:00
deployed,(none)
tags,
steps,


In [9]:
pipeline.deploy()

 ok


name,statsmodels-test-364c7a5c-8754-48c8-aaf0-2cc055a8d869
created,2022-09-22 14:33:00.562180+00:00
last_updated,2022-09-22 14:34:20.076489+00:00
deployed,True
tags,
steps,statsmodels-test-statmodels


### Run Inference

Once the pipeline is running, we can submit our data to the pipeline and return our results.  Once finished, we will undeploy the pipeline to return the resources back to the cluster.

In [10]:
pipeline.status()

{'status': 'Running',
 'details': None,
 'engines': [{'ip': '10.244.0.25',
   'name': 'engine-76f4b45db6-wkhjg',
   'status': 'Running',
   'reason': None,
   'pipeline_statuses': {'pipelines': [{'id': 'statsmodels-test-364c7a5c-8754-48c8-aaf0-2cc055a8d869',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'statsmodels-test-postprocess',
      'version': '8ac74f31-2fd6-4f97-a01b-6f7068d8e5c3',
      'sha': '142b39c639ea99b79001ed267d42d29785c4c299fc37d482f693dc072ec64df6',
      'status': 'Running'},
     {'name': 'statsmodels-test-statmodels',
      'version': '2f3261e6-cefd-426b-a309-5ec2a9f8ecda',
      'sha': '3125d2bd33fbf223d8af6e8d1add555483f343f2ea4260f9fbd22f2f72350932',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.3.16',
   'name': 'engine-lb-67c854cc86-dd7rq',
   'status': 'Running',
   'reason': None}]}

In [11]:
results = pipeline.infer_from_file('bike_day_eval_engine.json')
results

[InferenceResult({'check_failures': [],
  'elapsed': 600,
  'model_name': 'statsmodels-test-postprocess',
  'model_version': '8ac74f31-2fd6-4f97-a01b-6f7068d8e5c3',
  'original_data': {'holiday': [0, 0, 0, 0, 0, 0, 0],
                    'temp': [0.317391,
                             0.365217,
                             0.415,
                             0.54,
                             0.4725,
                             0.3325,
                             0.430435],
                    'windspeed': [0.184309,
                                  0.203117,
                                  0.209579,
                                  0.231017,
                                  0.368167,
                                  0.207721,
                                  0.288783],
                    'workingday': [1, 1, 1, 1, 0, 0, 1]},
  'outputs': [{'Float': {'data': [-0.7701932787895203,
                                  -0.15543800592422485,
                                  0.3652

In [12]:
assert results[0].data()[0].shape == (7, 1)

In [13]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ...................................... ok


name,statsmodels-test-364c7a5c-8754-48c8-aaf0-2cc055a8d869
created,2022-09-22 14:33:00.562180+00:00
last_updated,2022-09-22 14:34:20.076489+00:00
deployed,False
tags,
steps,statsmodels-test-statmodels
